# SWIS-100 "Single Run" notebook
## Set up run (interactive/single-shot)

In [ ]:
import os
import numpy as np
import pandas as pd
import swis100 as swis

In [ ]:
display(swis.solar_pu_raw)
display(swis.wind_pu_raw)
display(swis.elec_load_data_raw)
display(swis.transport_load_data_raw)
display(swis.when2heat_data)
display(swis.assumptions_raw)

In [ ]:
run_config = {}

run_config['solver_name'] = 'cbc'

run_config['assumptions_year'] = 2030 # Used to select projected nominal technology costs

# https://www.ecb.europa.eu/stats/policy_and_exchange_rates/euro_reference_exchange_rates/html/eurofxref-graph-usd.en.html
# Change from 7 July 2019 to 8 July 2020
# Minimum (20 March 2020): 1.0707 - Maximum (9 March 2020): 1.1456 - Average: 1.1058
run_config['usd_to_eur'] = (1.0/1.1058)

run_config['Nyears'] = 1
run_config['snapshot_interval'] = 12 # hours
run_config['elec_load_year_start'] = 2014
# Available year(s) for variable electricity load data: 2014-2019 inclusive
run_config['transport_load_year_start'] = 2014
# Available year(s) for transport load data: 1991-2017 inclusive 
run_config['heat_year_start'] = 2010
# Available year(s) for low temperature heat data: 2008-2013 inclusive 
run_config['weather_year_start'] = 2010
# Available year(s) for solar and wind weather data: solar 1985-2015 inclusive, wind 1980-2016

run_config['delta_CO2_atm_max (MtCO2)'] = 0.0 # Net removal over the run, MtCO2

# If want test on constant electricity load, set this True
run_config['constant_elec_load_flag'] = False
if run_config['constant_elec_load_flag'] :
    run_config['constant_elec_load (GW)'] = 1.0
else :
    run_config['elec_load_scope']="IE"

# If want test on constant surface transport load, set this True
run_config['constant_surface_transport_load_flag'] = False
if run_config['constant_surface_transport_load_flag'] :
    run_config['constant_surface_transport_load (GW)'] = 1.0

# If want test on constant air transport load, set this True
run_config['constant_air_transport_load_flag'] = False
if run_config['constant_air_transport_load_flag'] :
    run_config['constant_air_transport_load (GW)'] = 1.0

# If want test on constant low temperature heating load, set this True
run_config['constant_lo_temp_heat_load_flag'] = False
if run_config['constant_lo_temp_heat_load_flag'] :
    run_config['constant_lo_temp_heat_load (GW)'] = 1.0

run_config['nuclear_SMR_min_p (GW)'] = 0.0
run_config['nuclear_SMR_max_p (GW)'] = 0.0 # No nuclear

# Set nominal VRE marginal costs to contol dispatch priority. Lower cost gets higher
# priority). Set all to non-zero so that curtailment/dispatch down at source is 
# preferred over discarding energy from storage (though presumably as long as there are
# *any* fixed costs of storage, this would be avoided by minimising storage size anyway).
run_config['solar_marginal_cost'] =0.03 # €/MWh
run_config['onshore_wind_marginal_cost'] =0.02 # €/MWh
run_config['offshore_wind_marginal_cost'] =0.01 # €/MWh

run_config['offshore_wind_min_p (GW)'] = 0.0
run_config['offshore_wind_max_p (GW)'] = +np.inf
#run_config['offshore_wind_max_p (GW)'] = 20.0

run_config['onshore_wind_min_p (GW)'] = 0.0 # Config var does still need to be defined...
#run_config['onshore_wind_min_p (GW)'] = 4.2548+1.2763
    # Sep 2020 installed capacity (all onshore)
    # IE: 4.2548, NI: 1.2763
    # per eirgrid "System and Renewable Data Summary Report"
    # http://www.eirgridgroup.com/site-files/library/EirGrid/System-and-Renewable-Data-Summary-Report.xlsx
#run_config['onshore_wind_max_p (GW)'] = +np.inf
run_config['onshore_wind_max_p (GW)'] = 8.2 
    # 2030 ambition in CAP-2019 (RES-E 70%) - not necessarily upper limit

run_config['solar_min_p (GW)'] = 0.0
#run_config['solar_max_p (GW)'] = +np.inf
#run_config['solar_max_p (GW)'] = 1.5 
    # 2030 ambition in CAP-2019, via NDP (RES-E 55%) - not necessarily upper limit
run_config['solar_max_p (GW)'] = 1.5*2.0 # Speculative: x2 the 2030 CAP-2019 target as ~ultimate limit? 

#run_config['IC_min_p (GW)'] = 0.5+0.5 # EWIC + Moyle
run_config['IC_min_p (GW)'] = 0.0 # Config var does still need to be defined...

#run_config['IC_max_p (GW)'] = +np.inf # Unlimited IC
run_config['IC_max_p (GW)'] = 0.5+0.5+0.8 # EWIC + Moyle + Celtic
#run_config['IC_max_p (GW)'] = 0.5+0.5 # EWIC + Moyle
#run_config['IC_max_p (GW)'] = 0.0 # No IC

#run_config['IC_max_e (TWh)'] = +np.inf
run_config['IC_max_e (TWh)'] = (
    run_config['IC_max_p (GW)'] * 14.0 * 12.0)/1.0e3 # 14 days max IC buffer

run_config['Battery_max_p (MW)'] = +np.inf
run_config['Battery_max_e (MWh)'] = +np.inf

run_config['H2_electrolysis_tech'] = 'default'
#run_config['H2_electrolysis_tech'] = 'Nicola-NEL'

run_config['H2_electrolysis_max_p (GW)'] = +np.inf
run_config['H2_CCGT_max_p (GW)'] = +np.inf
run_config['H2_OCGT_max_p (GW)'] = +np.inf

run_config['H2_storage_tech'] = 'salt cavern'
#run_config['H2_storage_tech'] = 'rock cavern'
#run_config['H2_storage_tech'] = 'steel tank'

run_config['H2_store_max_e (TWh)'] = +np.inf

run_config['BEV_min_p (GW)'] = 0.0
run_config['BEV_max_p (GW)'] = +np.inf
run_config['FCEV_min_p (GW)'] = 0.0
run_config['FCEV_max_p (GW)'] = +np.inf

run_config['ASHP_min_p (GW)'] = 0.0
run_config['ASHP_max_p (GW)'] = +np.inf
run_config['H2_boiler_min_p (GW)'] = 0.0
run_config['H2_boiler_max_p (GW)'] = +np.inf

display(run_config)
#print(run_config)

## Solve the system (interactive/single-shot)

In [ ]:
network = swis.solve_network(run_config)

In [ ]:
run_stats=swis.gather_run_stats(run_config, network)
with pd.option_context('display.max_rows', None): # Display ALL rows (no ellipsis)
    display(run_stats)

## Some one-off plotting

### "Tolerance" note!

For some reason (solver tolerance?) some solution values that should be strictly 
positive or negative may be infinitesimally (< 10e-10) of the other sign. This will cause 
`df.plot(stacked=True)` to throw an error (requires all values to have same sign, positive or 
negative). So we do a somewhat ugly `.round(10)` kludge to fix it...

### Set up plot period

(NB: interactive plotting may be *sloooow* for large time windows!)

In [ ]:
Nyears = int(run_config['Nyears'])
weather_year_start = int(run_config['weather_year_start'])
weather_year_end = weather_year_start + (Nyears - 1)

plt_start = F"{weather_year_start}-01-01"
plt_stop = F"{weather_year_end}-12-31"

### Define consistent colours for specific timeseries variables

In [ ]:
# matplotlib named colors:
# https://matplotlib.org/3.1.0/gallery/color/named_colors.html
colors = {
    'ASHP' : 'maroon',
    'BEV' : 'purple',
    'CO2_atm_store' : 'red',
    'CO2_conc_store' : 'green',
    'DAC' : 'darkslategrey',
    'FT' : 'mediumpurple',
    'FCEV' : 'yellow',
    'H2 CCGT' : 'red',
    'H2 OCGT' : 'pink',
    'H2 electrolysis' : 'magenta',
    'H2 store' : 'green',
    'H2 store to bus' : 'green',
    'H2 store from bus' : 'green',
    'H2_boiler' : 'rosybrown',
    'Links p_gain (HP RE)' : 'yellow',
    'Links p_loss' : 'yellow',
    'Links p_load (DAC)' : 'darkslategrey',
    'Storage charge' : 'green',
    'Storage discharge' : 'green',
    'battery charge' : 'gray',
    'battery discharge' : 'gray',
    'battery storage' : 'gray',
    'ic-export' : 'green',
    'ic-import' : 'green',
    'lo-temp-heat-demand' : 'aqua',
    'local-elec-demand' : 'black',
    'local-elec-demand' : 'black',
    'nuclear-SMR' : 'aqua', 
    'offshore wind' : 'cornflowerblue',
    'onshore wind' : 'royalblue', 
    'remote-elec-grid-buffer' : 'magenta',
    'solar' : 'yellow', 
    'surface-transport-demand' : 'aqua',
    'air-transport-demand' : 'rosybrown'
}


### System-wide energy "balance"

Here we separate all "primary" energy sources (pypsa `Generators`), all "final" energy consumption (pypsa `Loads`, plus any designated "load" element associated with `Links` - currently only DAC), instanteous aggregate flow to/from storage (charge/discharge), and "losses". In this system, losses only arise in certain `Links`...

In [ ]:
# Exclude non-energy (CO2!) store flows
store_p = network.stores_t.p.drop(['CO2_atm_store','CO2_conc_store'],axis='columns')
# Calculate instantaneous aggregate flow in/out of storage (charge/discharge)
store_flow = store_p.sum(axis='columns')
#display(store_flow)
store_discharge = store_flow.clip(lower=0.0).rename('Storage discharge')
#display(store_discharge)
store_charge = store_flow.clip(upper=0.0).rename('Storage charge')
#display(store_charge)

links_p_gain = network.links_t.p_gain.sum(axis='columns').rename('Links p_gain (HP RE)')
#display(links_p_gain)
links_p_loss = network.links_t.p_loss.sum(axis='columns').rename('Links p_loss')
#display(links_p_loss)
links_p_load = network.links_t.p_load.sum(axis='columns').rename('Links p_load (DAC)') 
#display(links_p_load)

positive = pd.concat(
    [network.generators_t.p, links_p_gain, store_discharge],
    axis='columns').round(10) # pd.DataFrame
#display(positive)   

negative = pd.concat(
    [-network.loads_t.p, -links_p_load, links_p_loss, store_charge],
    axis='columns').round(10) # pd.DataFrame
#display(negative)

In [ ]:
# System *must* now "balance"!?
balance = positive.sum(axis=1) + negative.sum(axis=1) # MW
#display(balance)
#display(balance.max())
#display(balance.min())

assert(abs(balance<0.1).all()) # MW

In [ ]:
import matplotlib.pyplot as plt

# For non-interactive plots use this magic:
#%matplotlib inline
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
#plt.rcParams["figure.figsize"] = (16,8)

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 
plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

positive_max=positive.sum(axis=1).max()
negative_min=negative.sum(axis=1).min()

#print([load_max,demand_max,gen_max])

# Set y_lim top extra large (*2.5) to make space for the legend...
positive.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                            ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in positive.columns])

negative.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                           ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in negative.columns])

#ax.set_ylim(1.2*demand_max, 2.0*gen_max)
#ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("System balance (inflow is +ve, outflow/loss is -ve) [MW]")
ax.legend(ncol=3,loc="upper left")

#fig.tight_layout()
#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

### Balance at bus: `local_elec_grid` 

In [ ]:
gens = ["nuclear-SMR", "onshore wind", "offshore wind", "solar"]
loads = ["local-elec-demand"]
in_links = ["ic-import", "H2 CCGT", "H2 OCGT","battery discharge"]
out_links = ["ic-export", "battery charge", "H2 electrolysis", "BEV", "ASHP", "DAC"]

positive = pd.concat((network.generators_t.p[gens],
                      -network.links_t.p1[in_links]),
                     axis=1).round(10)
negative = pd.concat((-network.loads_t.p[loads],
                      -network.links_t.p0[out_links]),
                     axis=1).round(10)

In [ ]:
# bus must balance instantaneously!
balance = positive.sum(axis=1) + negative.sum(axis=1)
#display(balance)
#display(balance.max())
#display(balance.min())

assert(abs(balance<0.1).all()) # MW

In [ ]:
import matplotlib.pyplot as plt

# For non-interactive plots use this magic:
#%matplotlib inline
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
#plt.rcParams["figure.figsize"] = (16,8)

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 
plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

positive_max=positive.sum(axis='columns').max() # pd.Dataframe
negative_min=negative.sum(axis='columns').min() # pd.Dataframe

# Set y_lim top extra large (*2.5) to make space for the legend...
positive.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                            ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in positive.columns])

negative.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                           ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in negative.columns])

#ax.set_ylim(1.2*demand_max, 2.0*gen_max)
#ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("Dispatch (generation is +ve, demand is -ve) [MW]")
ax.legend(ncol=3,loc="upper left")

#fig.tight_layout()
#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

### Balance at bus: `surface_transport_final` 


In [ ]:
positive = -network.links_t.p1[["BEV","FCEV"]].round(10) # pd.Dataframe
#display(positive)
        
negative = -network.loads_t.p['surface-transport-demand'].round(10) # pd.Series
#display(negative_load)

In [ ]:
# bus must balance instantaneously!
balance = positive.sum(axis=1) + negative
#display(balance)
#display(balance.max())
#display(balance.min())

assert(abs(balance<0.1).all()) # MW

In [ ]:
import matplotlib.pyplot as plt

# For non-interactive plots use this magic:
#%matplotlib inline
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
#plt.rcParams["figure.figsize"] = (16,8)

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 
plt.rcParams["figure.figsize"] = (8,4) 

fig, ax = plt.subplots()

# Set y_lim top extra large (*2.5) to make space for the legend...

positive_max=positive.sum(axis='columns').max() # pd.Dataframe
negative_min=negative.min() # pd.Series

positive.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                            ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in positive.columns])

negative.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                           ylim=(1.2*negative_min, 2.5*positive_max),
                            color=colors['surface-transport-demand'])

ax.set_ylabel("Surface Transport [MW]")
ax.legend(ncol=3,loc="upper left")

#fig.tight_layout()
#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

### Balance at bus: `lo_temp_heat` 


In [ ]:
positive = -network.links_t.p1[["ASHP","H2_boiler"]].round(10) # pd.Dataframe
#display(positive)
        
negative = -network.loads_t.p['lo-temp-heat-demand'].round(10) # pd.Series
#display(negative)

In [ ]:
# bus must balance instantaneously!
balance = positive.sum(axis=1) + negative
#display(balance)
#display(balance.max())
#display(balance.min())

assert(abs(balance<0.1).all()) # MW

In [ ]:
import matplotlib.pyplot as plt

# For non-interactive plots use this magic:
#%matplotlib inline
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
#plt.rcParams["figure.figsize"] = (16,8)

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 
plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

# Set y_lim top extra large (*2.5) to make space for the legend...

positive_max=positive.sum(axis='columns').max() # pd.Dataframe
negative_min=negative.min() # pd.Series

positive.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                            ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in positive.columns])

negative.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                           ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[negative.name]])

ax.set_ylabel("Low temperature heat [MW]")
ax.legend(ncol=3,loc="upper left")

#fig.tight_layout()
#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

### Balance at bus: `H2` 


In [ ]:
store_flow = network.stores_t.p["H2 store"]
#display(store_flow)
store_flow_to_bus = store_flow.clip(lower=0.0).rename("H2 store to bus")
#display(store_flow_to_bus)
store_flow_from_bus = store_flow.clip(upper=0.0).rename("H2 store from bus")
#display(store_flow_from_bus)

positive = pd.concat(
    [-network.links_t.p1["H2 electrolysis"].round(10),
    store_flow_to_bus.round(10)],axis='columns') # pd.DataFrame
#display(positive)   

negative = pd.concat(
    [-network.links_t.p0[[ "FCEV", "H2 CCGT", "H2 OCGT", "H2_boiler", "FT"]].round(10),
    store_flow_from_bus.round(1)],axis='columns') # pd.DataFrame
#display(negative)

In [ ]:
# bus must balance instantaneously!
balance = positive.sum(axis=1) + negative.sum(axis=1)
#display(balance)
#display(balance.max())
#display(balance.min())

assert(abs(balance<0.1).all()) # MW

In [ ]:
import matplotlib.pyplot as plt

# For non-interactive plots use this magic:
#%matplotlib inline
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
#plt.rcParams["figure.figsize"] = (16,8)

# For interactive plots use this magic (REQUIRES ipympl package installed!):
%matplotlib widget 
plt.rcParams["figure.figsize"] = (8,4)

fig, ax = plt.subplots()

# Set y_lim top extra large (*2.5) to make space for the legend...

positive_max=positive.sum(axis='columns').max() # pd.Dataframe
negative_min=negative.sum(axis='columns').min() # pd.Series

positive.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                            ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in positive.columns])

negative.loc[plt_start:plt_stop].plot(kind="area",stacked=True,ax=ax,linewidth=0,
                           ylim=(1.2*negative_min, 2.5*positive_max),
                            color=[colors[i] for i in negative.columns])

ax.set_ylabel("Low temperature heat [MW]")
ax.legend(ncol=3,loc="upper left")

#fig.tight_layout()
#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

### Energy levels in Store components 

In [ ]:
import matplotlib.pyplot as plt

# For non-interactive plots use this magic:
%matplotlib inline
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
plt.rcParams["figure.figsize"] = (16,8)

# For interactive plots use this magic (REQUIRES ipympl package installed!):
#%matplotlib widget 

fig, ax = plt.subplots()

storage = network.stores_t.e[["remote-elec-grid-buffer","H2 store",
                              "battery storage"]].round(10)
#display(storage)

storage.loc[plt_start:plt_stop].plot(kind="line",stacked=True,ax=ax,linewidth=1,
                              color=[colors[i] for i in storage.columns])

e_max = storage.sum(axis=1).max()
e_min = storage.sum(axis=1).min()

#ax.set_ylim([demand_max,gen_max])
#print([demand_max,gen_max])
ax.set_ylim([0.0, 1.2*e_max])
ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("Storage [MWh]")
ax.legend(ncol=3,loc="upper left")

fig.tight_layout()

#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

### CO$_2$ levels in Store components 

In [ ]:
import matplotlib.pyplot as plt

# For non-interactive plots use this magic:
%matplotlib inline
# Set plotsize in notebook
# https://www.mikulskibartosz.name/how-to-change-plot-size-in-jupyter-notebook/
plt.rcParams["figure.figsize"] = (16,8)

# For interactive plots use this magic (REQUIRES ipympl package installed!):
#%matplotlib widget 

fig, ax = plt.subplots()

storage = network.stores_t.e[["CO2_atm_store",
                              "CO2_conc_store"]]
#display(storage)

storage.loc[plt_start:plt_stop].plot(kind="line",ax=ax,linewidth=1,
                              color=[colors[i] for i in storage.columns])

e_max = storage.max().sum()
#display(e_max)
e_min = storage.min().sum()
#display(e_min)

#ax.set_ylim([demand_max,gen_max])
#print([demand_max,gen_max])
ax.set_ylim([e_min, 1.2*e_max])
ax.set_xlim([plt_start,plt_stop])
ax.set_ylabel("Storage [tCO2]")
ax.legend(ncol=3,loc="upper left")

fig.tight_layout()

#fig.savefig("img/{}-{}-{}-{}.png".format(ct,scenario,start,stop),dpi=100)

## Dump (volatile) run data to (persistent) file storage (just in case...)


In [ ]:
run_id = 'scratch' # Tailor this as desired!
run_dir='runs/single-run/'+run_id
os.makedirs(run_dir,exist_ok=True)

#network.export_to_netcdf(run_dir+'/network.nc') # Uncomment to save all network object data
run_config_series=pd.Series(run_config, dtype=object, name=run_id)
run_config_df=run_config_series.to_frame().transpose()
run_config_df.to_excel(run_dir+'/run_config.ods')
run_stats.to_excel(run_dir+'/run_stats.ods')

## \<scratchpad\>

**Delete before public release!**

In [ ]:
mwhr_per_gj = 0.2778 # https://www.seai.ie/data-and-insights/seai-statistics/conversion-factors/
mwhr_per_mj = mwhr_per_gj / 1.0e3
syn_fuel_mj_per_kg = 44.0 # https://en.wikipedia.org/wiki/Aviation_fuel#Energy_content
syn_fuel_mj_per_t = syn_fuel_mj_per_kg * 1.0e3
syn_fuel_mwh_per_t = syn_fuel_mj_per_t * mwhr_per_mj
print(F'syn_fuel_mwh_per_t: {syn_fuel_mwh_per_t:6.2f}')

syn_fuel_shadow_price_per_mwh = 540.93 # €/MWh
syn_fuel_shadow_price_per_t = syn_fuel_shadow_price_per_mwh*syn_fuel_mwh_per_t
print(F'syn_fuel_shadow_price_per_t: {syn_fuel_shadow_price_per_t:6.0f}')

# vs Jan 2021 conventional jet fuel price (exc. taxes) ~ US$467/t??
jet_fuel_price_2021_per_t = 467.0*run_config['usd_to_eur']
print(F'jet_fuel_price_2021_per_t: {jet_fuel_price_2021_per_t:6.0f}')
syn_fuel_price_multiplier = syn_fuel_shadow_price_per_t/jet_fuel_price_2021_per_t
print(F'syn_fuel_price_multiplier: {syn_fuel_price_multiplier:6.0f}')


In [ ]:
display(network.links_t.p2['Aircraft'])

In [ ]:
display(network.links_t.p2['DAC'])

In [ ]:
display(network.stores_t.p['CO2_atm_store'])

In [ ]:
display(network.stores_t.p['syn_fuel_store'])

In [ ]:
display(network.stores_t.e['syn_fuel_store'])

In [ ]:
display(network.stores.loc['syn_fuel_store',:])

## \</scratchpad\>